In [1]:
# import libraries
import pandas as pd
import polars as pl
import io
import requests
import json
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

from src.config import CONNECTION_URI
from supabase import create_client, Client

In [10]:
hist_2021 = pl.read_csv("../data/viirs-yearly-summary/viirs-snpp_2021.csv")
hist_2022 = pl.read_csv("../data/viirs-yearly-summary/viirs-snpp_2022.csv")
hist_2023 = pl.read_csv("../data/viirs-yearly-summary/viirs-snpp_2023.csv")

display(hist_2021.head(3), hist_2022.head(3), hist_2023.head(3))

latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
f64,f64,f64,f64,f64,str,i64,str,str,str,i64,f64,f64,str,i64
-1.454696,127.433228,341.71,0.66,0.73,"""1/1/2021""",400,"""N""","""VIIRS""","""n""",1,287.14,9.83,"""D""",3
-3.989202,122.100601,338.51,0.5,0.49,"""1/1/2021""",542,"""N""","""VIIRS""","""n""",1,289.81,4.83,"""D""",0
-2.577868,121.379036,350.77,0.47,0.48,"""1/1/2021""",542,"""N""","""VIIRS""","""n""",1,285.65,35.66,"""D""",2


latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
f64,f64,f64,f64,f64,str,i64,str,str,str,i64,f64,f64,str,i64
-1.255306,121.606781,330.97,0.53,0.42,"""2022-01-01""",500,"""N""","""VIIRS""","""n""",1,275.37,4.03,"""D""",0
-3.865005,136.417984,332.43,0.42,0.61,"""2022-01-01""",500,"""N""","""VIIRS""","""n""",1,283.68,2.68,"""D""",3
-1.257126,121.605209,332.45,0.53,0.42,"""2022-01-01""",500,"""N""","""VIIRS""","""n""",1,274.84,6.87,"""D""",0


latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
f64,f64,f64,f64,f64,str,i64,str,str,str,str,f64,f64,str
0.451,109.06416,334.46,0.42,0.38,"""1/1/2023""",559,"""N""","""VIIRS""","""n""","""2.0NRT""",288.34,27.04,"""D"""
-1.32986,113.40453,342.43,0.41,0.37,"""1/1/2023""",559,"""N""","""VIIRS""","""n""","""2.0NRT""",279.44,7.98,"""D"""
-0.84505,116.93285,341.42,0.41,0.45,"""1/1/2023""",559,"""N""","""VIIRS""","""n""","""2.0NRT""",286.1,9.12,"""D"""


There's different length of columns between 2021-2022 and 2023 dataset. In this case, it's because the dataset for 2023 hasn't been processed yet since it's a "2.0 NRT" version. To combine them into one big dataframe, we need to add "type" column into hist_2023 and cast the columns into one consistent type.

In [11]:
hist_2021 = hist_2021.select(
    pl.col("latitude").cast(pl.Float32),
    pl.col("longitude").cast(pl.Float32),
    pl.col("brightness").cast(pl.Float32),
    pl.col("scan").cast(pl.Float32),
    pl.col("track").cast(pl.Float32),
    pl.col("acq_date").str.strptime(pl.Date, "%m/%d/%Y"),
    pl.col("acq_time").cast(pl.Int16),
    pl.col("satellite").cast(pl.Utf8),
    pl.col("instrument").cast(pl.Utf8),
    pl.col("confidence").cast(pl.Utf8),
    pl.col("version").cast(pl.Utf8),
    pl.col("bright_t31").cast(pl.Float32),
    pl.col("frp").cast(pl.Float32),
    pl.col("daynight").cast(pl.Utf8),
    pl.col("type").cast(pl.Int16),
)

hist_2022 = hist_2022.select(
    pl.col("latitude").cast(pl.Float32),
    pl.col("longitude").cast(pl.Float32),
    pl.col("brightness").cast(pl.Float32),
    pl.col("scan").cast(pl.Float32),
    pl.col("track").cast(pl.Float32),
    pl.col("acq_date").str.strptime(pl.Date, "%Y-%m-%d"),
    pl.col("acq_time").cast(pl.Int16),
    pl.col("satellite").cast(pl.Utf8),
    pl.col("instrument").cast(pl.Utf8),
    pl.col("confidence").cast(pl.Utf8),
    pl.col("version").cast(pl.Utf8),
    pl.col("bright_t31").cast(pl.Float32),
    pl.col("frp").cast(pl.Float32),
    pl.col("daynight").cast(pl.Utf8),
    pl.col("type").cast(pl.Int16),
)

hist_2023 = hist_2023.with_columns(
    type = pl.lit(None)
)

hist_2023 = hist_2023.select(
    pl.col("latitude").cast(pl.Float32),
    pl.col("longitude").cast(pl.Float32),
    pl.col("brightness").cast(pl.Float32),
    pl.col("scan").cast(pl.Float32),
    pl.col("track").cast(pl.Float32),
    pl.col("acq_date").str.strptime(pl.Date, "%m/%d/%Y"),
    pl.col("acq_time").cast(pl.Int16),
    pl.col("satellite").cast(pl.Utf8),
    pl.col("instrument").cast(pl.Utf8),
    pl.col("confidence").cast(pl.Utf8),
    pl.col("version").cast(pl.Utf8),
    pl.col("bright_t31").cast(pl.Float32),
    pl.col("frp").cast(pl.Float32),
    pl.col("daynight").cast(pl.Utf8),
    pl.col("type").cast(pl.Int16),
)

display(hist_2021.head(3), hist_2022.head(3), hist_2023.head(3))

latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
f32,f32,f32,f32,f32,date,i16,str,str,str,str,f32,f32,str,i16
-1.454696,127.433228,341.709991,0.66,0.73,2021-01-01,400,"""N""","""VIIRS""","""n""","""1""",287.140015,9.83,"""D""",3
-3.989202,122.100601,338.51001,0.5,0.49,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",289.809998,4.83,"""D""",0
-2.577868,121.379036,350.769989,0.47,0.48,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",285.649994,35.66,"""D""",2


latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
f32,f32,f32,f32,f32,date,i16,str,str,str,str,f32,f32,str,i16
-1.255306,121.606781,330.970001,0.53,0.42,2022-01-01,500,"""N""","""VIIRS""","""n""","""1""",275.369995,4.03,"""D""",0
-3.865005,136.417984,332.429993,0.42,0.61,2022-01-01,500,"""N""","""VIIRS""","""n""","""1""",283.679993,2.68,"""D""",3
-1.257126,121.605209,332.450012,0.53,0.42,2022-01-01,500,"""N""","""VIIRS""","""n""","""1""",274.839996,6.87,"""D""",0


latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
f32,f32,f32,f32,f32,date,i16,str,str,str,str,f32,f32,str,i16
0.451,109.064163,334.459991,0.42,0.38,2023-01-01,559,"""N""","""VIIRS""","""n""","""2.0NRT""",288.339996,27.040001,"""D""",null
-1.32986,113.404533,342.429993,0.41,0.37,2023-01-01,559,"""N""","""VIIRS""","""n""","""2.0NRT""",279.440002,7.98,"""D""",null
-0.84505,116.932854,341.420013,0.41,0.45,2023-01-01,559,"""N""","""VIIRS""","""n""","""2.0NRT""",286.100006,9.12,"""D""",null


In [12]:
concatenated_hist = pl.concat([hist_2021, hist_2022, hist_2023])
concatenated_hist = concatenated_hist.sort("acq_date")

concatenated_hist

latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
f32,f32,f32,f32,f32,date,i16,str,str,str,str,f32,f32,str,i16
-1.454696,127.433228,341.709991,0.66,0.73,2021-01-01,400,"""N""","""VIIRS""","""n""","""1""",287.140015,9.83,"""D""",3
-3.989202,122.100601,338.51001,0.5,0.49,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",289.809998,4.83,"""D""",0
-2.577868,121.379036,350.769989,0.47,0.48,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",285.649994,35.66,"""D""",2
0.487806,122.279358,333.049988,0.6,0.53,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",290.579987,5.19,"""D""",0
-1.373236,121.310432,335.929993,0.48,0.48,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",290.649994,5.3,"""D""",0
-6.041213,105.929779,333.209991,0.42,0.61,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",286.279999,4.21,"""D""",3
-2.159446,115.582207,328.700012,0.39,0.36,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",286.209991,2.28,"""D""",2
-0.923966,120.00058,331.200012,0.4,0.44,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",291.390015,2.58,"""D""",0
-2.577709,121.37899,345.339996,0.59,0.53,2021-01-01,1642,"""N""","""VIIRS""","""n""","""1""",290.700012,14.06,"""N""",2


Insert into supabase

In [4]:
query = "SELECT * FROM viirs_snpp_raw LIMIT 5"

pl.read_database_uri(query=query, uri=CONNECTION_URI)

id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
i64,f64,f64,f32,f32,f32,date,i32,str,str,str,str,f32,f32,str,i32
1,-1.454696,127.433228,341.709991,0.66,0.73,2021-01-01,400,"""N""","""VIIRS""","""n""","""1""",287.140015,9.83,"""D""",3
2,-3.989202,122.100601,338.51001,0.5,0.49,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",289.809998,4.83,"""D""",0
3,-2.577868,121.379036,350.769989,0.47,0.48,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",285.649994,35.66,"""D""",2
4,0.487806,122.279358,333.049988,0.6,0.53,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",290.579987,5.19,"""D""",0
5,-1.373236,121.310432,335.929993,0.48,0.48,2021-01-01,542,"""N""","""VIIRS""","""n""","""1""",290.649994,5.3,"""D""",0


In [7]:
concatenated_hist.write_database(table_name="viirs_snpp_raw",  connection=CONNECTION_URI, if_exists="append")